<a href="https://colab.research.google.com/github/Nihitha8819/image-classification/blob/main/image_classification_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,MaxPooling2D,Conv2D

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(64,kernel_size=(3,3),))

In [4]:
!pip install duckduckgo_search


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.8 MB/s eta 0:00:00


In [5]:
import os
import shutil
import requests
from duckduckgo_search import DDGS

def download_images(query, total, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    count = 0
    with DDGS() as ddgs:
        for result in ddgs.images(query, max_results=total*2):  # Fetch more in case of broken links
            if count >= total:
                break
            url = result['image']
            try:
                response = requests.get(url, timeout=5)
                if response.status_code == 200:
                    ext = url.split('.')[-1].split('?')[0]
                    if ext.lower() not in ['jpg', 'jpeg', 'png']:
                        ext = 'jpg'
                    filename = f"{query.replace(' ', '_')}_{count}.{ext}"
                    with open(os.path.join(target_dir, filename), 'wb') as f:
                        f.write(response.content)
                    count += 1
            except:
                continue
    print(f"[✓] Downloaded {count} images for {query}")

# Download 20 images each
download_images("lion", 20, "raw/lion")
download_images("elephant", 20, "raw/elephant")

# Organize into training and testing
for animal in ['lion', 'elephant']:
    images = os.listdir(f'raw/{animal}')
    os.makedirs(f'dataset/training/{animal}', exist_ok=True)
    os.makedirs(f'dataset/testing/{animal}', exist_ok=True)

    for i, img in enumerate(images):
        src = f'raw/{animal}/{img}'
        if i < 16:
            dst = f'dataset/training/{animal}/{img}'
        else:
            dst = f'dataset/testing/{animal}/{img}'
        shutil.copy(src, dst)

print("\n[✓] Dataset organized into training (16) and testing (4) folders.")


[✓] Downloaded 20 images for lion
[✓] Downloaded 20 images for elephant

[✓] Dataset organized into training (16) and testing (4) folders.


In [8]:
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
from tensorflow.  keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
    )
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_path  = r'/content/dataset/training'
test_path = r'/content/dataset/testing'
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    batch_size=8,
    class_mode='binary' #categorical
)
validation_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary' #categorical
)

Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [13]:
train_generator.class_indices

{'elephant': 0, 'lion': 1}

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #categorical_crossentropy

In [15]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5458 - loss: 1.1954 - val_accuracy: 0.5000 - val_loss: 0.6988
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 459ms/step - accuracy: 0.5125 - loss: 0.7003 - val_accuracy: 0.5000 - val_loss: 0.7002
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 446ms/step - accuracy: 0.4208 - loss: 0.7209 - val_accuracy: 0.5000 - val_loss: 0.6956
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 562ms/step - accuracy: 0.4167 - loss: 0.7099 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 626ms/step - accuracy: 0.6042 - loss: 0.6927 - val_accuracy: 0.5000 - val_loss: 0.6966
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 645ms/step - accuracy: 0.5542 - loss: 0.6892 - val_accuracy: 0.5000 - val_loss: 0.6939
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 532ms/step - accuracy: 0.5000 - loss: 0.6938 - val_accuracy: 0.5000 - val_loss: 0.6962
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 494ms/step - accuracy: 0.2875 - loss: 0.7226 - val_accuracy: 0.5000 - val_

In [16]:
model.save('elephant-lion.h5')